# 9-class specific DL-LA method

- Input: hamming weight
- Output:

In [83]:
import numpy as np
from keras import Sequential
from keras.layers import Dense
from keras.optimizer_v2.adam import Adam
from scipy.stats import stats
from tensorflow.python.keras.utils.np_utils import to_categorical
from tqdm import tqdm

from src.data.ascad import AscadRandomKey
from src.dlla.hw import plot_predictions, dlla_p_gradient, hamming_weight_prediction, fetch_traces

from src.dlla.wegener import wegener_p_gradient, binomial_test
from src.tools.balance import balance
from src.trace_set.transform import reduce_fixed_fixed, reduce_fixed_random

In [2]:
ascad = AscadRandomKey().default

X, Y, X_ATT, Y_ATT = fetch_traces(ascad)

In [94]:
class HWMethodType:
    mlp_hw = "mlp_hw"


class AbstractMethod:
    default_parameters = None

    def confidence_value(self, x_att, y_att):
        raise NotImplementedError()

    def p_gradient(self, x_att, y_att):
        raise NotImplementedError()

def t_test(a, b):
    return stats.ttest_ind(a, b, equal_var=False)[1]

class DllaHW(AbstractMethod):
    default_parameters = {
        'activation': 'relu',
        'optimizer': Adam(learning_rate=0.001),
        'losses': 'categorical_crossentropy',
        'batch_size': 150,
        'epochs': 5,
        'hidden_layers': [100] * 4
    }

    def __init__(self, x, y, params=default_parameters):
        model = Sequential()
        for ix, nodes in enumerate(params['hidden_layers']):
            if ix == 0:
                layer = Dense(nodes, activation=params['activation'], input_shape=(x.shape[1],))
            else:
                layer = Dense(nodes, activation=params['activation'])

            model.add(layer)

        model.add(Dense(9, activation='softmax'))
        model.compile(optimizer=params['optimizer'], loss=params['losses'], metrics=['accuracy'])

        model.fit(x, y, shuffle=True, batch_size=params['batch_size'], epochs=params['epochs'], verbose=False)

        self.model = model

    def confidence_value(self, x_att, y_att):
        is_high = np.argmax(y_att, axis=1).astype(bool)
        nr = min(len(x_att), len(y_att))

        l4 = hamming_weight_prediction(self.model, x_att[np.where(~is_high)], nr)
        g4 = hamming_weight_prediction(self.model, x_att[np.where(is_high)], nr)

        return t_test(l4, g4)

    def plot_predictions(self, x_att, y_att):
        plot_predictions(self.model, x_att, y_att)

    def p_gradient(self, x_att, y_att):
        dlla_p_gradient(self.model, x_att, y_att)

In [80]:
PROF_TRACES = None
ATT_TRACES = 1000

In [81]:
METHOD9 = DllaHW(X[:PROF_TRACES], Y[:PROF_TRACES])
METHOD9.confidence_value(*reduce_fixed_random(X_ATT[:ATT_TRACES], Y_ATT[:ATT_TRACES]))

8.077944774283927e-150

0.03340596636284827

In [95]:
class DllaWegener(AbstractMethod):
    def __init__(self, x, y):
        trace_len = x.shape[1]

        model = Sequential()
        model.add(Dense(120, activation='relu', input_shape=(trace_len,)))
        model.add(Dense(90, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(2, activation='softmax'))

        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics='accuracy')

        model.fit(x, y, shuffle=True, batch_size=150, epochs=5, verbose=False)

        self.model = model

    def confidence_value(self, x_att, y_att):
        predictions = self.model.predict(x_att).argmax(axis=1)
        labels = y_att.argmax(axis=1)
        correct = np.array(predictions == labels)

        return binomial_test(len(x_att), correct.sum())

    def p_gradient(self, x_att, y_att):
        return wegener_p_gradient(self.model, x_att, y_att)

METHOD2 = DllaWegener(*reduce_fixed_fixed(X[:PROF_TRACES], Y[:PROF_TRACES]))
CONFIDENCE = METHOD2.confidence_value(*reduce_fixed_fixed(X_ATT[:ATT_TRACES], Y_ATT[:ATT_TRACES]))


In [99]:
for _ in tqdm(range(1000)):
    METHOD2 = DllaWegener(*reduce_fixed_fixed(X[:PROF_TRACES], Y[:PROF_TRACES]))
    CONFIDENCE_FF = METHOD2.confidence_value(*reduce_fixed_fixed(X_ATT[:ATT_TRACES], Y_ATT[:ATT_TRACES]))
    CONFIDENCE_FR = METHOD2.confidence_value(*reduce_fixed_random(X_ATT[:ATT_TRACES], Y_ATT[:ATT_TRACES]))

    with open("method2-fix-fix-1000att.txt", "a") as file:
        file.write(f"{CONFIDENCE_FF}\n")

    with open("method2-fix-random-1000att.txt", "a") as file:
        file.write(f"{CONFIDENCE_FR}\n")

    METHOD9 = DllaHW(X[:PROF_TRACES], Y[:PROF_TRACES])
    CONFIDENCE_FF = METHOD9.confidence_value(*reduce_fixed_fixed(X_ATT[:ATT_TRACES], Y_ATT[:ATT_TRACES]))
    CONFIDENCE_FR = METHOD9.confidence_value(*reduce_fixed_random(X_ATT[:ATT_TRACES], Y_ATT[:ATT_TRACES]))

    with open("method9-fix-fix-1000att.txt", "a") as file:
        file.write(f"{CONFIDENCE_FF}\n")

    with open("method9-fix-random-1000att.txt", "a") as file:
        file.write(f"{CONFIDENCE_FR}\n")

# TODO: is this on normalized traces? - yes: fetch_traces()

 32%|███▏      | 319/1000 [1:49:50<3:54:28, 20.66s/it]


KeyboardInterrupt: 